In [1]:
%config IPCompleter.greedy=True
import pandas as pd
import os
from unicodedata import normalize
import pandas as pd
import nltk.data
import re, string, unicodedata
import nltk
import num2words
from num2words import num2words

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
from textblob import TextBlob

arrayHeaders = ["Area", "Dimension", "Respuesta"]
data = pd.read_csv("Data/example.csv", header= None, names = arrayHeaders)

In [2]:
arrayAreas= []
for area in data["Area"]:
    area = area.split('\n')
    area = area[2]
    area = area.replace(':es: ','')
    area = area.replace('"','')
    area = area.replace('\r','')
    arrayAreas.append(area)
arrayAreas = pd.Series(arrayAreas)

In [3]:
arrayDimensiones = []
for dimension in data["Dimension"]:
    dimension = dimension.split('\n')
    dimension = dimension[2]
    dimension = dimension.replace(':es: ','')
    dimension = dimension.replace('"','')
    dimension = dimension.replace('\r','')
    arrayDimensiones.append(dimension)
    
arrayDimensiones = pd.Series(arrayDimensiones)

In [4]:
posAnswers = []
negAnswers = []

for resp in data["Respuesta"]:
    resp = resp.split('\n')
    resp_pos = resp[2].replace('       positive: ','')
    resp_pos = resp_pos.replace("'",'')
    resp_pos = resp_pos.replace('\r','')
    posAnswers.append(resp_pos)
    if len(resp)==4:
        resp_neg = resp[3].replace('       negative: ','')
        resp_neg = resp_neg.replace("'",'')
        resp_neg = resp_neg.replace('\r','')
        negAnswers.append(resp_neg)
    else:
        negAnswers.append('')
        
posAnswers = pd.Series(posAnswers)
negAnswers = pd.Series(negAnswers)

In [5]:
concatenaciónPositivas = pd.concat([arrayAreas,arrayDimensiones, posAnswers], axis = 1)
concatenaciónNegativas = pd.concat([arrayAreas,arrayDimensiones, negAnswers], axis = 1)
df = pd.concat([concatenaciónPositivas,concatenaciónNegativas],ignore_index=True).rename(columns = pd.Series(arrayHeaders))

In [6]:
def remplaceSimbols(token):
    token = re.sub(r'[^\w\s]', ' ', token)
    return token

df["Respuesta"] = df["Respuesta"].apply(remplaceSimbols)

In [7]:
dataTestTokenized = df.copy()
dataTestTokenized["Respuesta"] = dataTestTokenized["Respuesta"].apply(nltk.word_tokenize)

In [8]:
stop = []
stop = stopwords.words('spanish')

In [9]:
def verifyStopWords(tokens):
    for token in tokens:
        if token.lower() in stop:
            tokens.pop(tokens.index(token))
    return tokens

In [10]:
dataTestStop = dataTestTokenized.copy()
dataTestStop["Respuesta"] = dataTestStop["Respuesta"].apply(verifyStopWords)

In [11]:
dfMoreWords = dataTestStop.copy()
dfMoreWords = dfMoreWords[dfMoreWords['Respuesta'].map(len) > 3].copy()
dfMoreWords = dfMoreWords.dropna()

In [12]:
def  verifyNormalize(tokens):
    for token in tokens:
        if token.isdigit()== False:
            token2 = re.sub(r'[^\w\s]', ' ', token)
            if token != token2:
                print(token+" pasa a "+token2)
    return tokens

def normalizeTokens(tokens):
    for token in tokens:
        pos = tokens.index(token)
        if token.isdigit():
            tokens[pos] = num2words(token, lang='es')
        else:
            token = re.sub(r'[^\w\s]', ' ', token)
            if token != ' ' or token != '':
                tokens[pos] = token.lower()
                #tokens[pos] = unicodedata.normalize('NFKD', token.lower()).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            else:
                tokens.pop(pos)
    return tokens

def verifyLang(tokens):
    count = 0
    for token in tokens:
        if token.isdigit() == False:
            b = TextBlob(token)
            if b.detect_language() == "es":
                count = count + 1
    if len(tokens)-2 < count:
        return ''
    else:
        return tokens
        

In [13]:
dfNormalized = dfMoreWords.copy()
dfNormalized["Respuesta"] = dfNormalized['Respuesta'].apply(normalizeTokens)

DataFinalFlujo1 = dfNormalized.copy()
DataFinalFlujo1["Respuesta"] = DataFinalFlujo1["Respuesta"].apply(verifyStopWords)

In [14]:
lemmaDiccionario = {}
with open('Herramientas/lemma.txt', 'rb') as fichero:
    datos = (fichero.read().decode('utf8').replace(u'\r', u'').split(u'\n'))
    datos = ([avance.split(u'\t') for avance in datos])
for avance in datos:
   if len(avance) >1:
      lemmaDiccionario[avance[1]] = avance[0]
        
def lemmatize(word):
   return lemmaDiccionario.get(word, word + u'')
   
def lemmatize_words(words):
    new_words = []
    for palabra in words:
        new_word = lemmatize(palabra)
        new_words.append(new_word)
    return new_words

In [15]:
DataFinalFlujo1["Respuesta"] = DataFinalFlujo1["Respuesta"].apply(lemmatize_words)

#Fin Flujo 1

In [16]:
from nltk.stem import SnowballStemmer
Stemmer_spanish = SnowballStemmer('spanish')
def stemWordsArray(tokens):
    for token in tokens:
        tokens[tokens.index(token)] = Stemmer_spanish.stem(token)
    return tokens

DataFinalFlujo2 = DataFinalFlujo1.copy()
DataFinalFlujo2['Respuesta'] = DataFinalFlujo2['Respuesta'].apply(stemWordsArray)

In [ ]:
DataFinalFlujo2.to_csv("Data/Flujo2.csv", sep=";", index=False)